In [1]:
from openai import OpenAI
import anthropic
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
# DEEPSEEK_MODEL = "openrouter/deepseek/deepseek-chat"
DEEPSEEK_MODEL = "openrouter/deepseek/deepseek-r1"

In [3]:
O1_MODEL = 'o1-mini'

o1_prompt = """
You are an expert researcher with deep expertise in finance, legal, and tax matters.
The first input you will receive will be a complex Research task that needs to be carefully reasoned through to solve. 
Your task is to review the challenge, conduct thorough research, and create a detailed plan to analyze information, assess implications, and provide comprehensive insights.


You will have access to an LLM agent that is responsible for executing the plan that you create and will return results.


The LLM agent has access to the following functions:
    - search_web(Question)
        - This function performs a web search and returns relevant information based on the provided query
    - HumanInput(Question)
          - This functions Get the Input from user with a specific question or Prompt. Use it only its absolutely necessary.
        
When creating a plan for the LLM to execute, break your instructions into a logical, step-by-step order, using the specified format:
    - **Main actions are numbered** (e.g., 1, 2, 3).
    - **Sub-actions are lettered** under their relevant main actions (e.g., 1a, 1b).
        - **Sub-actions should start on new lines**
    - **Specify conditions using clear 'if...then...else' statements** (e.g., 'If the financial statement shows a profit, then...').
    - **For actions that require using one of the above functions defined**, write a step to call a function using backticks for the function name (e.g., `call the fetch_context function`).
        - Ensure that the proper input arguments are given to the model for instruction. There should not be any ambiguity in the inputs.
    - **The last step** in the instructions should always be calling the `instructions_complete` function. This is necessary so we know the LLM has completed all of the instructions you have given it.
    - **Detailed steps** The plan generated must be extremely detailed and thorough with explanations at every step.
Use markdown format when generating the plan with each step and sub-step.

Please find the scenario below.
"""

In [4]:
import os
from litellm import completion

def call_o1(scenario):
    prompt = f"""
    {o1_prompt}
        
    Scenario:
    {scenario}

    Please provide the next steps in your plan.
    """
    # client = OpenAI()
    # response = client.chat.completions.create(
    #     model=O1_MODEL,
    #     messages=[{'role': 'user', 'content': prompt}]
    # )
    response = completion(
        model=DEEPSEEK_MODEL,
        messages=[{'role': 'user', 'content': prompt}]
    )
    plan = response.choices[0].message.content
    return plan

c:\Users\nandh\AppData\Local\Programs\Python\Python310\lib\site-packages\pydantic\_internal\_config.py:341: UserWarning: Valid config keys have changed in V2:
* 'fields' has been removed
  warnings.warn(message, UserWarning)


In [5]:
def append_message(message_list, message):
    message_list.append(message)
    # # Optionally, print the message for immediate feedback
    message_type = message.get('type', '')
    if message_type == 'status':
        print(message['message'])
    elif message_type == 'plan':
        print("\nPlan:\n", message['content'])
    elif message_type == 'assistant':
        print("\nAssistant:\n", message['content'])
    elif message_type == 'function_call':
        print(f"\nFunction call: {message['function_name']} with arguments {message['arguments']}")
    elif message_type == 'function_response':
        print(f"\nFunction response for {message['function_name']}: {message['response']}")
    else:
        # Handle any other message types or default case
        print(message.get('content', ''))

In [6]:
import os
import re
from openai import OpenAI

def search_web(query):
    """
    Function to search the web using Perplexity AI API and return the answer
    with inline citation links instead of numbered references.
    
    Args:
        query (str): The search query to be processed
        
    Returns:
        str: The response content with inline links replacing citations like [1], [2], etc.
    """
    client = OpenAI(api_key=os.environ["PERPLEXITYAI_API_KEY"], base_url="https://api.perplexity.ai")
    
    messages = [
        {
            "role": "system",
            "content": (
                "You are an artificial intelligence assistant and you need to "
                "answer like a Data collection engine with as much information as possible."
            ),
        },
        {   
            "role": "user",
            "content": query,
        },
    ]

    # chat completion without streaming
    response = client.chat.completions.create(
        model="sonar",
        messages=messages,
    )
    
    # Extract the main content and citations list
    content = response.choices[0].message.content
    citations = response.citations  # This is typically a list of URLs or references

    # Function to replace each [number] with the actual URL from the citations list
    def replace_citation_with_link(match):
        # Extract the digit inside the brackets, e.g., [1] -> "1"
        citation_num_str = match.group(0)[1:-1]
        citation_idx = int(citation_num_str) - 1
        
        # Safety check: If citation index is out of range, just return the original match
        if citation_idx < 0 or citation_idx >= len(citations):
            return match.group(0)
        
        # Replace with actual link in parentheses (or any other format you prefer)
        return f"({citations[citation_idx]})"
    
    # Replace patterns like [1], [2], etc., with the corresponding link
    # NOTE: If the AI sometimes returns combined citations like [1][2], you can
    # handle them by capturing consecutive groups or by running the re.sub multiple times.
    content_with_links = re.sub(r'\[\d+\]', replace_citation_with_link, content)

    # If the model sometimes produces multiple consecutive citations (e.g. [1][2]),
    # you can run another pass or use a more complex regex. For most cases, a single
    # pass will suffice if the model typically returns separated citations.
    # Example of a second pass if needed:
    # content_with_links = re.sub(r'\]\[', '], [', content_with_links)

    return content_with_links

def instructions_complete(final_report):
    return f"Final Report: {final_report}"

function_mapping = {
    'search_web': search_web,
    'instructions_complete': instructions_complete
}

In [7]:
TOOLS = [
    {
        "name": "search_web",
        "description": "function performs a web search and returns relevant information based on the provided query",
        "input_schema": {
            "type": "object",
            "properties": {
                "query": {
                    "type": "string",
                    "description": "The search query to be processed"
                }
            },
            "required": ["query"],
        },
    },
    {
        "name": "instructions_complete",
        "description": "Function should be called when we have completed ALL of the instructions.",
        "input_schema": {
            "type": "object",
            "properties": {
                "final_report": {
                    "type": "string",
                    "description": "Final Report based on the analysis."
                }
            },
            "required": ["final_report"],
        }
    }
]

In [8]:
system_prompt = """
You are a helpful assistant responsible for executing the policy on handling Deep research Tasks. 
Your task is to follow the policy exactly as it is written and perform the necessary actions.

You must explain your decision-making process across various steps.

# Steps

1. **Read and Understand Policy**: Carefully read and fully understand the given policy on Deep research Task.
2. **Identify the exact step in the policy**: Determine which step in the policy you are at, and execute the instructions according to the policy.
3. **Decision Making**: Briefly explain your actions and why you are performing them.
4. **Action Execution**: Perform the actions required by calling any relevant functions and input parameters. 

POLICY:
"""

In [9]:
def parse_response_blocks(response):
    """
    Parse the response from Anthropic API and identify text and tool blocks.
    Returns a tuple of (has_tool_block, text_content, tool_info)
    """
    content = response.content
    text_content = None
    tool_info = None
    has_tool_block = False

    for block in content:
        if hasattr(block, 'type'):
            if block.type == 'text':
                text_content = block.text
            elif block.type == 'tool_use':
                has_tool_block = True
                tool_info = {
                    'name': block.name,
                    'input': block.input,
                    'id': block.id
                }

    return has_tool_block, text_content, tool_info

In [10]:
def call_antropic(message_list, plan):
    policy_prompt = system_prompt

    client = anthropic.Anthropic()
    messages = [
        {'role': 'user', 'content': plan}
    ]
    i = 1
    while True:
        response = client.messages.create(
                model="claude-3-5-sonnet-20241022",
                max_tokens=1024,
                system=policy_prompt,
                tools=TOOLS,
                messages=messages
        )
        
        messages.append({"role": "assistant", "content": response.content})
    
        has_tool, text_content, tool_info = parse_response_blocks(response)
        print(f"Iteration-{i}\nHasTool = {has_tool}\ntext_content\n{text_content}\nTool_info = {tool_info}")
        if text_content is not None:
            append_message(message_list, {"type": "assistant", "content": text_content})
        
        if tool_info['name'] in function_mapping:
            res = function_mapping[tool_info['name']](**tool_info['input'])
            print(f"Tool {tool_info['name']} Response:\n", res)
            if tool_info['name'] == "instructions_complete":
                break
        else:
            print(f"Unknown tool: {tool_info['name']}")
            
        messages.append({"role": "user", "content": [{
            "type": "tool_result",
            "tool_use_id": tool_info['id'],
            "content": res
        }]})
        i += 1
    return messages
    

In [11]:
def process_scenario(message_list, scenario):
    append_message(message_list, {'type': 'status', 'message': 'Generating plan...'})

    plan = call_o1(scenario)

    append_message(message_list, {'type': 'plan', 'content': plan})

    append_message(message_list, {'type': 'status', 'message': 'Executing plan...'})

    messages = call_antropic(message_list, plan)

    append_message(message_list, {'type': 'status', 'message': 'Processing complete.'})

    return messages

In [12]:
scenario = "What are the typical factors that affect the Indian stock market's performance, and what investment approaches do experienced investors generally follow in the Indian market context?"
messages = process_scenario([], scenario)

Generating plan...

Plan:
 **Plan to Analyze Factors Affecting the Indian Stock Market and Investment Approaches**  

---

### **1. Research Economic Factors Influencing the Indian Stock Market**  
1a. **Assess macroeconomic indicators**  
   - `Call the search_web function` with the query: *"Current GDP growth rate in India and its impact on stock market trends 2023"*.  
   - Compare GDP trends over the last 5 years to identify correlations with market performance.  
   - **Condition**: If GDP growth is slowing, check for government stimulus measures (e.g., infrastructure spending).  

1b. **Evaluate inflation and interest rates**  
   - `Call the search_web function` with the query: *"RBI repo rate changes 2022–2023 and equity market reactions"*.  
   - Analyze how repo rate hikes/cuts impacted liquidity, corporate borrowing costs, and sectoral performance.  
   - **Condition**: If inflation remains above RBI targets, assess sectors vulnerable to rising input costs (e.g., FMCG, manuf

In [13]:
for m in messages:
    print(m)

{'role': 'user', 'content': '**Plan to Analyze Factors Affecting the Indian Stock Market and Investment Approaches**  \n\n---\n\n### **1. Research Economic Factors Influencing the Indian Stock Market**  \n1a. **Assess macroeconomic indicators**  \n   - `Call the search_web function` with the query: *"Current GDP growth rate in India and its impact on stock market trends 2023"*.  \n   - Compare GDP trends over the last 5 years to identify correlations with market performance.  \n   - **Condition**: If GDP growth is slowing, check for government stimulus measures (e.g., infrastructure spending).  \n\n1b. **Evaluate inflation and interest rates**  \n   - `Call the search_web function` with the query: *"RBI repo rate changes 2022–2023 and equity market reactions"*.  \n   - Analyze how repo rate hikes/cuts impacted liquidity, corporate borrowing costs, and sectoral performance.  \n   - **Condition**: If inflation remains above RBI targets, assess sectors vulnerable to rising input costs (e.

In [24]:
def generate_outline(messages, scenario):
    system_prompt = """
    You are a research assistant specialized in creating focused research outlines.
    Your role is to:
    1. Analyze the provided scenario/question
    2. Extract key research themes directly related to the question
    3. Generate a hierarchical outline covering only the relevant themes
    4. Use markdown format with clear heading levels (###, -, *)
    5. Ensure every heading directly addresses components of the research question

    Do not include:
    - Methodological steps (like data collection or verification)
    - Process-related headings (like "Final Steps")
    - Any content not directly answering the research question
    """
    
    user_prompt = f"""
    Based on the following research scenario and messages, create a focused outline that directly addresses the key components of the question.

    <Research>
        {scenario}
    </Research>
    
    <message>
        {messages}
    </message>

    REQUIREMENTS:
    1. Use markdown formatting
    2. Include only headings relevant to answering the scenario
    3. Structure as main headings (###) and subheadings (-)
    4. Ensure each heading connects to either:
    - Factors affecting Indian stock market performance
    - Investment approaches of experienced Indian investors

    OUTPUT FORMAT:
    ### [Main Topic]
    - [Subtopic]
    - [Subtopic]
    """
    
    response = completion(
        model="gpt-4o-mini",
        messages=[{"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}]
    )

    return response.choices[0].message.content

In [25]:
outline = generate_outline(messages, scenario)
print(outline)

### Factors Affecting the Indian Stock Market's Performance
- Economic Factors
  - Macroeconomic Indicators
  - Inflation and Interest Rates
  - Corporate Earnings
- Political and Policy Developments
  - Government Stability and Reforms
  - Fiscal Policy and Taxation Changes
- Global Factors and External Risks
  - Foreign Institutional Investor (FII) Activity
  - Commodity Price Fluctuations
  - Global Indices and Geopolitical Risks
- Market-Specific Dynamics
  - Sectoral Performance Drivers
  - Retail Investor Behavior

### Investment Approaches of Experienced Investors
- Fundamental vs. Technical Analysis
- Asset Allocation Trends
- Risk Management Techniques


In [29]:
from litellm import completion

def get_final_report(messages, scenario, outline):
    system_prompt = """
    You are a professional report writer specializing in detailed research analysis and structured reporting.
    Your primary responsibility is to generate comprehensive reports that precisely follow given outline structures
    while providing in-depth analysis of research data.

    Key Responsibilities:
    - Strictly adhere to provided outline structures
    - Generate detailed content for each outline section and subsection
    - Ensure thorough coverage of all points in the outline
    - Maintain consistent depth of analysis across all sections
    - Support all findings with specific evidence from research data
    """

    user_prompt = f"""
    Generate a comprehensive, detailed report following the exact structure of the provided outline.
    Each section must be thoroughly developed with supporting evidence from the research data.

    <Research Context>
    {scenario}
    </Research Context>

    <Research Structure>
    {outline}
    </Research Structure>

    <Research Data>
    {messages}
    </Research Data>

    Report Requirements:
    1. STRICTLY FOLLOW THE PROVIDED OUTLINE:
    - Generate content for every section and subsection in the outline
    - Maintain the exact hierarchy and organization specified
    - Use consistent heading levels that match the outline structure
    - Ensure no outline points are skipped or merged

    2. DETAIL AND EVIDENCE:
    - Provide extensive detail for each outline point
    - Include multiple supporting examples from research data
    - Quote relevant passages from the research materials
    - Analyze each point thoroughly before moving to the next

    3. FORMATTING AND STRUCTURE:
    - Use markdown headers that match outline levels (# for main sections, ## for subsections, etc.)
    - Include transitional text between sections to maintain flow
    - Format quotes and evidence appropriately
    - Maintain consistent depth across all sections

    4. ANALYSIS REQUIREMENTS:
    - Provide data-driven insights for each section
    - Include metrics and quantitative analysis where applicable
    - Draw connections between related findings
    - Support each conclusion with specific evidence

    Generate the report now, beginning with the first outline point and maintaining strict adherence to the outline structure.
    """
    response = completion(
        model="openrouter/minimax/minimax-01",
        messages=[
            {'role': 'system', 'content': system_prompt},
            {'role': 'user', 'content': user_prompt}
        ]
    )               
    return response.choices[0].message.content


def clean_markdown(content):
    return content.replace('```markdown', '').replace('```', '')

In [30]:
final_report = get_final_report(messages, scenario, outline)
clean_final_report = clean_markdown(final_report)
print(clean_final_report)

# Indian Stock Market Analysis Report 2023-2024

## Factors Affecting the Indian Stock Market's Performance

### Economic Factors

#### Macroeconomic Indicators

India's macroeconomic environment has a profound impact on its stock market performance. The country's GDP growth rate is a critical indicator of economic health and investor confidence.

- **GDP Growth Trends**: 
  - For the financial year 2023-24, India's real GDP growth rate was estimated at 8.2%, a significant increase from the 7.0% growth in the previous fiscal year. This robust growth was driven by strong performance in sectors such as manufacturing and construction, which contributed to a positive outlook for the stock market.
  - However, the GDP growth slowed to 5.4% in the September quarter of 2024, the lowest since the December quarter of 2022. This slowdown was attributed to global economic uncertainties and domestic challenges, which tempered market enthusiasm despite the strong annual growth.

- **Impact on Stock